# Running 2_PT_Senseflux_intersectionpoint for the whole year each two month

In [1]:
import xarray as xr
import os
FVCOM_DIR = '/mnt/hydroglg/Data/External_Models/Outputs/GLCFS/LakeHuron/2024/'


In [ ]:
all_files = sorted([f for f in os.listdir(FVCOM_DIR) if f.endswith(".nc")])

print("Earliest file:", all_files[0])

In [ ]:

import cftime
from netCDF4 import num2date
filename = 'lmhofs.t00z.20241101.fields.n006.nc'

ds = xr.open_dataset(FVCOM_DIR + filename, decode_times=False)
time_var = ds.variables['time']

# Extract the values and metadata properly
time_vals = time_var[:]
units = time_var.attrs.get('units', None)
calendar = time_var.attrs.get('calendar', 'standard')

if units is None:
    raise ValueError("The 'units' attribute is missing from the time variable.")

# Convert to datetime
converted_time = num2date(time_vals, units=units, calendar=calendar)

print("Start time:", converted_time[0])
print("End time:", converted_time[-1])


In [ ]:
for fname in sorted(os.listdir(FVCOM_DIR)):
    if fname.endswith('.nc'):
        ds = xr.open_dataset(os.path.join(FVCOM_DIR, fname), decode_times=False)
        print(f"{fname}: {ds['time'].values[0]} to {ds['time'].values[-1]}")
        ds.close()

In [ ]:
from netCDF4 import Dataset


import os
import shutil
from netCDF4 import Dataset


# Directory containing your (renamed) FVCOM files
BASE_DIR = "/mnt/hydroglg/Data/External_Models/Outputs/GLCFS/LakeHuron/rename"

# File glob patterns to include. Adjust if needed.
# If you've renamed nos.* files to lmhofs.*, one pattern is enough.
PATTERNS = ("lmhofs.")  # we match startswith

# Where to move problem files
MISSING_SUBDIR = os.path.join(BASE_DIR, "missing_Itime")

# Safety: do a dry run first. Set to False to actually move.
DRY_RUN = True
# --------------------------------------------------------------------------


def has_Itime(path):
    """Return True if NetCDF file contains variable 'Itime'."""
    try:
        with Dataset(path, "r") as ds:
            return "Itime" in ds.variables
    except Exception as e:
        print(f"!! ERROR reading {os.path.basename(path)}: {e}")
        return False  # treat unreadable as missing


def main():
    os.makedirs(MISSING_SUBDIR, exist_ok=True)

    files = sorted(
        f for f in os.listdir(BASE_DIR)
        if f.endswith(".nc") and f.startswith(PATTERNS)
    )

    if not files:
        print("No matching NetCDF files found. Check BASE_DIR/PATTERNS.")
        return

    keep = []
    move = []

    for fname in files:
        fpath = os.path.join(BASE_DIR, fname)
        if has_Itime(fpath):
            keep.append(fname)
        else:
            move.append(fname)

    print("\nScan complete.")
    print(f"  Files with Itime: {len(keep)}")
    print(f"  Files MISSING Itime: {len(move)}")

    if not move:
        print("Nothing to move. You're good!")
        return

    # Show list of problem files
    print("\nProblem files (missing Itime):")
    for fname in move:
        print("  ", fname)

    if DRY_RUN:
        print("\nDRY RUN: No files moved. Set DRY_RUN = False to move them.")
        return

    # Move them
    for fname in move:
        src = os.path.join(BASE_DIR, fname)
        dst = os.path.join(MISSING_SUBDIR, fname)
        print(f"Moving {fname} -> missing_Itime/")
        shutil.move(src, dst)

    print("\nDone. Re-scan to confirm.")


if __name__ == "__main__":
    main()

In [ ]:
from netCDF4 import Dataset, num2date
import os
import glob

def get_earliest_valid_datetime(data_dir):
    file_list = sorted(glob.glob(os.path.join(data_dir, "*.nc")))

    all_times = []
    for f in file_list:
        try:
            print(f"Checking: {f}")
            ds = Dataset(f)
            if 'time' not in ds.variables:
                ds.close()
                continue

            time_var = ds.variables['time']
            if len(time_var[:]) == 0:
                ds.close()
                continue

            units = time_var.units
            calendar = getattr(time_var, 'calendar', 'standard')
            times = num2date(time_var[:], units=units, calendar=calendar)

            if isinstance(times, list) or hasattr(times, '__getitem__'):
                all_times.append(times[0])
            else:
                all_times.append(times)

            ds.close()

        except Exception as e:
            print(f"Skipping {f}: {e}")
            continue

    if not all_times:
        raise ValueError("No valid times found in any NetCDF file!")

    earliest = min(all_times)
    print(f"Earliest detected FVCOM time: {earliest}")
    return earliest.replace(microsecond=0)


In [2]:
import os
import glob
import nbformat
from nbconvert.preprocessors import ExecutePreprocessor
import datetime
import configparser
import subprocess
# FVCOM-specific visualization and utility tools
from pylag.processing.plot import FVCOMPlotter, create_figure, colourmap
from pylag.processing.utils import get_grid_bands
from pylag.grid_metrics import create_fvcom_grid_metrics_file

# Regridding, viewing, and garbage collection utilities
from pylag.regrid import regridder
from pylag.processing.ncview import Viewer
####
def run_pylag(start_datetime, end_datetime, config_file_name, out_dir, MODELLING_DIR, pylag_cfg_path):
    start_str = start_datetime.strftime('%Y-%m-%d %H:%M:%S')
    end_str = end_datetime.strftime('%Y-%m-%d %H:%M:%S')

    # Update config files
    update_datetime_in_config(config_file_name, start_str, end_str, out_dir)
    print(f"Updated start_datetime, end_datetime, and output_file in {config_file_name} and pylag.cfg.")
    print(f"Output file for this run: {start_str[:7]}_run.nc")

    # Change to modelling directory
    os.chdir(MODELLING_DIR)

    # Run the model
    run_command = f"python -m pylag.main -c {pylag_cfg_path}"
    print(f"Running model with command: {run_command}")

    try:
        result = subprocess.run(run_command, shell=True, check=True, capture_output=True, text=True)
        print("Model run completed.")
        print(f"stdout: {result.stdout}")
        print(f"stderr: {result.stderr}")
    except subprocess.CalledProcessError as e:
        print(f"⚠️ Skipping run from {start_str} to {end_str} due to error.")
        print(f"stdout: {e.stdout}")
        print(f"stderr: {e.stderr}")
        
def files_available_for_range(start_datetime, end_datetime, FVCOM_DIR, stem='lmhofs'):
    # Generate all expected file patterns based on daily data assumption
    current = start_datetime
    while current <= end_datetime:
        # Match either file format: lmhofs.t06z.YYYYMMDD.fields.nXXX.nc or lmhofs.fields.nXXX.YYYYMMDD.tXXz.nc
        date_str1 = current.strftime('%Y%m%d')
        pattern1 = f"{FVCOM_DIR}/{stem}.t*z.{date_str1}.fields.n*.nc"
        pattern2 = f"{FVCOM_DIR}/{stem}.fields.n*.{date_str1}.t*z.nc"

        files1 = glob.glob(pattern1)
        files2 = glob.glob(pattern2)

        if not (files1 or files2):
            print(f"⚠️ No files found for date {current.date()}")
            return False

        current += datetime.timedelta(days=1)
    
    return True
# Function to generate start and end datetimes for each two-month period
def generate_date_ranges(year):
    # Adjust to create correct date ranges from August to December
    #months = [(2,3),(3,4),(4,5),(5,6),(6,7),(7,8),(8, 9),(9, 10),(10, 11),(11, 12)]
    months = [(9,10),(10,11),(11,12)]
    
    date_ranges = []
    for start_month, end_month in months:
        start_datetime = datetime.datetime(year, start_month, 1)
        
        # Special case for September: Set end date to 29th
        if start_month == 1:
            start_datetime = datetime.datetime(year, start_month, 1, 5, 00, 00)
        # if start_month == 8:
        #     start_datetime = datetime.datetime(year, start_month, 1, 6, 00, 00)
        # if start_month == 11:
        #     start_datetime = datetime.datetime(year, start_month, 1, 23, 19, 32)
        if end_month == 10:
            end_datetime = datetime.datetime(year, end_month, 30, 00, 00, 00)
        if end_month == 8:
            end_datetime = datetime.datetime(year, end_month, 31, 6, 00, 00)
        if end_month == 11:
            end_datetime = datetime.datetime(year, end_month, 30, 00, 00, 00)
        elif end_month == 12:
            end_datetime = datetime.datetime(year, end_month, 31, 16, 00, 00)
        else:
            end_datetime = datetime.datetime(year, end_month + 1, 1) - datetime.timedelta(seconds=1)
        
        date_ranges.append((start_datetime, end_datetime))
    
    return date_ranges

# Function to update the datetime and output filename in the config file
def update_datetime_in_config(config_file_name, start_str, end_str, out_dir):
    start_date = datetime.datetime.strptime(start_str, '%Y-%m-%d %H:%M:%S')
    end_date = datetime.datetime.strptime(end_str, '%Y-%m-%d %H:%M:%S')
    
    start_year = start_date.year
    start_month = start_date.month
    end_year = end_date.year
    end_month = end_date.month
    
    month_range = f"{start_date.strftime('%b')}{end_date.strftime('%b')}"
    output_filename = f"FVCOM_Huron_{start_year % 100}{end_year % 100}_{month_range}"

    # Read the config file and update the necessary values
    with open(config_file_name, 'r') as file:
        lines = file.readlines()

    with open(config_file_name, 'w') as file:
        for line in lines:
            if line.strip().startswith("start_datetime"):
                file.write(f"start_datetime = {start_str}\n")
            elif line.strip().startswith("end_datetime"):
                file.write(f"end_datetime = {end_str}\n")
            elif line.strip().startswith("output_file"):
                # Ensure the output file name is updated correctly
                file.write(f"output_file = %(out_dir)s/{output_filename}\n")
            else:
                file.write(line)

    # Now, update the pylag.cfg with the same changes
    pylag_cfg_path = os.path.join(out_dir, 'pylag.cfg')
    with open(pylag_cfg_path, 'w') as config_file:
        for line in lines:
            if line.strip().startswith("start_datetime"):
                config_file.write(f"start_datetime = {start_str}\n")
            elif line.strip().startswith("end_datetime"):
                config_file.write(f"end_datetime = {end_str}\n")
            elif line.strip().startswith("output_file"):
                config_file.write(f"output_file = %(out_dir)s/{output_filename}\n")
            else:
                config_file.write(line)

# Main script
def main():
    # Define directories and file paths
    config_file_path = '/home/abolmaal/modelling/FVCOM/Huron/config_files'
    config_file_name = os.path.join(config_file_path, 'Huron_Senseflux_Seasonal.cfg')
    MODELLING_DIR = '/home/abolmaal/modelling/FVCOM/Huron'
    out_dir = os.path.join(MODELLING_DIR, 'output')
    FVCOM_DIR = '/mnt/hydroglg/Data/External_Models/Outputs/GLCFS/LakeHuron/2024/'
    input_dir = os.path.join(MODELLING_DIR, 'input')
    grid_metrics_file_name = f'{input_dir}/gridfile/grid_metrics_huron_senseflux_Seasonal.nc'
    pylag_cfg_path = os.path.join(out_dir, 'pylag.cfg')

    # Create necessary directories
    os.makedirs(input_dir, exist_ok=True)
    os.makedirs(out_dir, exist_ok=True)

    # Initialize the configuration parser
    cf = configparser.ConfigParser()
    cf.read(config_file_name)

    # Set configuration parameters
    cf.set('OCEAN_DATA', 'data_dir', FVCOM_DIR)
    cf.set('OCEAN_DATA', 'grid_metrics_file', grid_metrics_file_name)
    cf.set('GENERAL', 'out_dir', out_dir)

    # Save the updated configuration to pylag.cfg
    with open(pylag_cfg_path, 'w') as config_file:
        cf.write(config_file)

    print(f"Updated configuration and saved to {pylag_cfg_path}")

    # Generate date ranges for the year
    year = 2024
    date_ranges = generate_date_ranges(year)

    # Loop through each date range and run the model
    for start_datetime, end_datetime in date_ranges:
        start_str = start_datetime.strftime('%Y-%m-%d %H:%M:%S')
        end_str = end_datetime.strftime('%Y-%m-%d %H:%M:%S')
        
        print(f"Running for: {start_str} to {end_str}")
        
        # # Update the datetime in the config file and pylag.cfg file
        # update_datetime_in_config(config_file_name, start_str, end_str, out_dir)
        # print(f"Updated start_datetime, end_datetime, and output_file in {config_file_name} and pylag.cfg.")
        
        # # Print the output filename to confirm it's unique for each run
        # print(f"Output file for this run: {start_str[:7]}_run.nc")  # Example: FVCOM_Huron_2323_JanFeb_run.nc
        
        # # Change to the modeling directory
        # os.chdir(MODELLING_DIR)

        # # Run the model using the updated pylag.cfg file with a new output path
        # run_command = f"python -m pylag.main -c {pylag_cfg_path}"
        # print(f"Running model with command: {run_command}")

        # # Execute the model with subprocess and capture output
        # try:
        #     result = subprocess.run(run_command, shell=True, check=True, capture_output=True, text=True)
        #     print("Model run completed.")
        #     print(f"stdout: {result.stdout}")
        #     print(f"stderr: {result.stderr}")
        # except subprocess.CalledProcessError as e:
        #     print(f"Error occurred while running the model:")
        #     print(f"stdout: {e.stdout}")
        #     print(f"stderr: {e.stderr}")
        #     print(f"Return code: {e.returncode}")
    if files_available_for_range(start_datetime, end_datetime, FVCOM_DIR):
        run_pylag(start_datetime, end_datetime, config_file_name, out_dir, MODELLING_DIR, pylag_cfg_path)
    else:
        print(f"⏭️ Skipping {start_datetime} to {end_datetime} due to missing input files.\n")


if __name__ == "__main__":
    main()

IndentationError: unexpected indent (file_reader.py, line 311)